In [2]:
import nltk
import random
from collections import Counter

# Завантажуємо необхідні ресурси NLTK
nltk.download('conll2002')

# 1. Завантаження корпусу CONLL 2002 (іспанська частина)
raw_train = nltk.corpus.conll2002.tagged_words('esp.train')
raw_test = nltk.corpus.conll2002.tagged_words('esp.testa')

# 2. ВИПРАВЛЕНА функція для перетворення даних у формат списку речень
def structure_sentences(tagged_words):
    """
    Структурує плаский список слів у список речень.
    Ця версія стійка до неконсистентності даних (кортежі з 2 або 3 елементів).
    """
    sentences = []
    current_sentence = []
    for item in tagged_words:
        # Перевіряємо, чи має елемент 3 частини (word, pos, ner)
        if len(item) == 3:
            word, pos, ner = item
        # чи лише 2 частини (word, pos)
        elif len(item) == 2:
            word, pos = item
        else:
            # Пропускаємо некоректні елементи
            continue

        current_sentence.append((word.lower(), pos))
        # Розділяємо речення за крапкою
        if word == '.':
            sentences.append(current_sentence)
            current_sentence = []

    # Додаємо останнє речення, якщо воно залишилось
    if current_sentence:
        sentences.append(current_sentence)
    return sentences

train_sents = structure_sentences(raw_train)
test_sents = structure_sentences(raw_test)

print(f"Загальна кількість речень у навчальній вибірці: {len(train_sents)}")
print(f"Загальна кількість речень у тестовій вибірці: {len(test_sents)}")
print("Приклад речення:", train_sents[0])

# 3. Функція для запису даних у файли (залишається без змін)
def write_tagged_sentences(sentences, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            if not sentence:
                continue
            for word, tag in sentence:
                f.write(f"{word}\t{tag}\n")
            f.write("\n")

# Записуємо навчальну та тестову вибірки у файли
write_tagged_sentences(train_sents, "conll_training.pos")
write_tagged_sentences(test_sents, "conll_test.pos")

# 4. Створення та збереження словника
word_counter = Counter(word for sentence in train_sents for word, tag in sentence)
vocab = {word for word, count in word_counter.items() if count >= 2}

with open("conll_vocab.txt", 'w', encoding='utf-8') as f:
    for word in sorted(list(vocab)):
        f.write(f"{word}\n")

print(f"\nРозмір словника (слова, що зустрічаються >= 2 рази): {len(vocab)}")

# 5. Створення файлу з тестовими словами
with open("conll_test_words.txt", 'w', encoding='utf-8') as f:
    for sentence in test_sents:
        if not sentence:
            continue
        for word, _ in sentence:
            f.write(f"{word}\n")
        f.write("\n")

print("Підготовка даних завершена. Файли збережено.")

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


Загальна кількість речень у навчальній вибірці: 7263
Загальна кількість речень у тестовій вибірці: 1640
Приклад речення: [('melbourne', 'NP'), ('(', 'Fpa'), ('australia', 'NP'), (')', 'Fpt'), (',', 'Fc'), ('25', 'Z'), ('may', 'NC'), ('(', 'Fpa'), ('efe', 'NC'), (')', 'Fpt'), ('.', 'Fp')]

Розмір словника (слова, що зустрічаються >= 2 рази): 12574
Підготовка даних завершена. Файли збережено.


In [3]:
from collections import defaultdict

def create_dictionaries(training_corpus_path, vocab):
    """
    Створює словники для частот емісій, переходів та тегів.
    """
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    prev_tag = '--s--'

    with open(training_corpus_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                transition_counts[(prev_tag, '--s--')] += 1
                prev_tag = '--s--'
                continue

            word, tag = line.split('\t')

            # У цій версії ми не обробляємо невідомі слова тут,
            # оскільки це буде зроблено пізніше в алгоритмі Вітербі.
            # Однак, для підрахунку емісій, ми можемо замінити їх на '--unk--'
            word_to_count = word if word in vocab else '--unk--'

            transition_counts[(prev_tag, tag)] += 1
            emission_counts[(tag, word_to_count)] += 1
            tag_counts[tag] += 1
            prev_tag = tag

    return emission_counts, transition_counts, tag_counts

# Створюємо словники на основі НОВИХ навчальних даних
emission_counts, transition_counts, tag_counts = create_dictionaries("conll_training.pos", vocab)

# Додаємо тег початку речення до лічильника тегів
tag_counts['--s--'] = len(train_sents)

print("Приклад частоти емісії:", list(emission_counts.items())[0])
print("Приклад частоти переходу:", list(transition_counts.items())[0])
print("Приклад частоти тега:", list(tag_counts.items())[0])

Приклад частоти емісії: (('NP', 'melbourne'), 2)
Приклад частоти переходу: (('--s--', 'NP'), 319)
Приклад частоти тега: ('NP', 1150)


In [4]:
import numpy as np

# Крок 3: Матриця переходів A
def create_transition_matrix(alpha, tag_counts, transition_counts):
    all_tags = sorted(tag_counts.keys())
    tag_to_idx = {tag: i for i, tag in enumerate(all_tags)}
    idx_to_tag = {i: tag for i, tag in enumerate(all_tags)}
    num_tags = len(all_tags)
    A = np.zeros((num_tags, num_tags))

    for i in range(num_tags):
        prev_tag = idx_to_tag[i]
        count_prev_tag = tag_counts[prev_tag]
        for j in range(num_tags):
            tag = idx_to_tag[j]
            count_transition = transition_counts.get((prev_tag, tag), 0)
            A[i, j] = (count_transition + alpha) / (count_prev_tag + alpha * num_tags)
    return A, tag_to_idx, idx_to_tag

alpha = 0.001
A, tag_to_idx, idx_to_tag = create_transition_matrix(alpha, tag_counts, transition_counts)
print("Розмір матриці переходів A:", A.shape)


# Крок 4: Матриця емісій B
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)
    all_words = sorted(list(vocab)) + ['--unk--']
    word_to_idx = {word: i for i, word in enumerate(all_words)}
    idx_to_word = {i: word for i, word in enumerate(all_words)}
    num_words = len(all_words)
    B = np.zeros((num_tags, num_words))

    for i in range(num_tags):
        tag = all_tags[i]
        count_tag = tag_counts[tag]
        for j in range(num_words):
            word = idx_to_word[j]
            count_emission = emission_counts.get((tag, word), 0)
            B[i, j] = (count_emission + alpha) / (count_tag + alpha * num_words)
    return B, word_to_idx, idx_to_word

B, word_to_idx, idx_to_word = create_emission_matrix(alpha, tag_counts, emission_counts, vocab)
print("Розмір матриці емісій B:", B.shape)

Розмір матриці переходів A: (60, 60)
Розмір матриці емісій B: (60, 12575)


In [5]:
# Крок 5: Алгоритм Вітербі
import math

def viterbi(sentence, tag_to_idx, word_to_idx, A, B):
    num_tags = len(tag_to_idx)
    T = len(sentence)
    viterbi_matrix = np.full((num_tags, T), -np.inf)
    backpointer = np.zeros((num_tags, T), dtype=int)
    start_tag_idx = tag_to_idx['--s--']
    first_word_idx = word_to_idx.get(sentence[0], word_to_idx['--unk--'])

    for tag_idx in range(num_tags):
        if A[start_tag_idx, tag_idx] > 0 and B[tag_idx, first_word_idx] > 0:
            viterbi_matrix[tag_idx, 0] = math.log(A[start_tag_idx, tag_idx]) + math.log(B[tag_idx, first_word_idx])

    for t in range(1, T):
        word_idx = word_to_idx.get(sentence[t], word_to_idx['--unk--'])
        for j in range(num_tags):
            max_prob = -np.inf
            best_prev_tag_idx = -1
            for i in range(num_tags):
                if A[i, j] > 0 and B[j, word_idx] > 0:
                    prob = viterbi_matrix[i, t-1] + math.log(A[i, j]) + math.log(B[j, word_idx])
                    if prob > max_prob:
                        max_prob = prob
                        best_prev_tag_idx = i
            viterbi_matrix[j, t] = max_prob
            backpointer[j, t] = best_prev_tag_idx

    best_path = []
    last_tag_idx = np.argmax(viterbi_matrix[:, T-1])
    best_path.append(idx_to_tag[last_tag_idx])
    for t in range(T-1, 0, -1):
        last_tag_idx = backpointer[last_tag_idx, t]
        best_path.insert(0, idx_to_tag[last_tag_idx])
    return best_path

# Крок 6: Оцінка точності
from tqdm import tqdm

def compute_accuracy(test_sents, tag_to_idx, word_to_idx, A, B):
    num_correct = 0
    total = 0
    for sentence_tags in tqdm(test_sents, desc="Оцінка точності"):
        if not sentence_tags: continue
        sentence = [word for word, tag in sentence_tags]
        gold_tags = [tag for word, tag in sentence_tags]
        predicted_tags = viterbi(sentence, tag_to_idx, word_to_idx, A, B)
        for pred_tag, gold_tag in zip(predicted_tags, gold_tags):
            if pred_tag == gold_tag:
                num_correct += 1
            total += 1
    return num_correct / total

accuracy = compute_accuracy(test_sents, tag_to_idx, word_to_idx, A, B)
print(f"\nТочність реалізованої моделі на тестовій вибірці (CONLL 2002): {accuracy:.4f}")


# Крок 7: Порівняння з NLTK
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger

most_common_tag = Counter(tag for sent in train_sents for word, tag in sent).most_common(1)[0][0]
default_tagger = DefaultTagger(most_common_tag)
unigram_tagger = UnigramTagger(train_sents, backoff=default_tagger)
bigram_tagger = BigramTagger(train_sents, backoff=unigram_tagger)
nltk_accuracy = bigram_tagger.accuracy(test_sents)

print(f"\nТочність NLTK BigramTagger: {nltk_accuracy:.4f}")
print(f"Точність нашої реалізації: {accuracy:.4f}")
print(f"Різниця: {abs(accuracy - nltk_accuracy):.4f}")

Оцінка точності: 100%|██████████| 1640/1640 [04:40<00:00,  5.84it/s]



Точність реалізованої моделі на тестовій вибірці (CONLL 2002): 0.9352

Точність NLTK BigramTagger: 0.9211
Точність нашої реалізації: 0.9352
Різниця: 0.0141
